### Create a grid-level population data
This code creates a grid-level population data by following steps : 
1. Create grid cells
2. Count the number of grid cells (*ngrid*) that fall into admin units (e.g. towns, counties, provinces)
3. Calculate population density as : total population / ngrid
4. Assign population density to grid cells

Last modified : Jan 14 2022 by Imryoung Jeong (neptune0118@gmail.com)

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

from shapely.geometry import Polygon

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

os.chdir("/Users/imryoung/Dropbox/")

In [ ]:
nl_dir = "./Nightlight_global/grid_original.shp"

adm_bnd_dir = "./gis_maps/census_township/census_township_1975/bnd_dong_00_1975_EPSG5179i.shp"
adm_idx = 'twn_cd_1975'

pop_dir = "./Project_protestantism/Data/statafile/Population/population_1960_V5.dta"

out_dir = "./output/insert_the_output_name_here.csv"

###### 1_Create grid cells
-----

In [ ]:
# Set the starting point from DMSP 
nl = gpd.read_file(nl_dir).to_crs('epsg:5179')

xmin, ymin, xmax, ymax = nl.total_bounds

length = 250

cols = list(np.arange(xmin, xmax+length, length))
rows = list(np.arange(ymin, ymax+length, length))

# Create grid cells
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+length, y), (x+length, y+length), (x, y+length)]))
        
grid = gpd.GeoDataFrame({'geometry':polygons})
grid['id'] = np.arange(len(grid))

# Create centroid of a grid cell
grid_centroid = grid[['id']].copy()
grid_centroid['geometry'] = grid['geometry'].centroid

grid_centroid = gpd.GeoDataFrame(grid_centroid, crs = 'epsg:5179', geometry='geometry')

###### 2_Count the number of grid cells that fall into each admin unit
----

In [ ]:

adm_bnd = gpd.read_file(adm_bnd_dir, encoding = 'utf-8').to_crs(epsg=5179)
adm_bnd.rename(columns={'adm_dr_cd':adm_idx}, inplace=True)

# Assign twn_cd to each point
pt = grid_centroid.copy()
pt = gpd.sjoin(pt, gdf, how='left', op='within')
pt = pt[['id','geometry',adm_idx]]

# Count the number of points in each region
ngrid = pt.dropna(subset = [adm_idx]).groupby([adm_idx]).size()

###### 3_Calculate the population density
----

In [ ]:
# Import admin-level population data
pop = pd.read_stata(pop_dir)

# Append the number of grids
pop['ngrid'] = pop[adm_idx].map(ngrid)
dta.head(5)

In [ ]:
# Calculate the population density
filter1 = [col for col in pop if col.startswith(("pop","ind_manu"))]
pop[filter1] = pop[filter1].div(pop['ngrid'].values, axis =0)

# Assign the density to each grid
grid_pop = pd.merge(pt, pop, on = adm_idx, how = 'left')

grid_pop[199995:200000]

In [ ]:
grid_pop.to_csv(out_dir)